In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [13]:
spark= SparkSession.builder \
.master("local[4]") \
.appName("RDD_Filter") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","3g") \
.getOrCreate()

sc = spark.sparkContext

In [14]:
retailRdd = sc.textFile("C:\\Users\\ferha\\Downloads\\OnlineRetail.csv")
retailRdd.take(5)

['InvoiceNo;StockCode;Description;Quantity;InvoiceDate;UnitPrice;CustomerID;Country',
 '536365;85123A;WHITE HANGING HEART T-LIGHT HOLDER;6;1.12.2010 08:26;2,55;17850;United Kingdom',
 '536365;71053;WHITE METAL LANTERN;6;1.12.2010 08:26;3,39;17850;United Kingdom',
 '536365;84406B;CREAM CUPID HEARTS COAT HANGER;8;1.12.2010 08:26;2,75;17850;United Kingdom',
 '536365;84029G;KNITTED UNION FLAG HOT WATER BOTTLE;6;1.12.2010 08:26;3,39;17850;United Kingdom']

In [15]:
firstLine=retailRdd.first()
firstLineRdd = sc.parallelize([firstLine])

In [16]:
retailRddWithoutHeader = retailRdd.subtract(firstLineRdd)

In [17]:
retailRddWithoutHeader.take(3)

['536367;84969;BOX OF 6 ASSORTED COLOUR TEASPOONS;6;1.12.2010 08:34;4,25;13047;United Kingdom',
 '536369;21756;BATH BUILDING BLOCK WORD;3;1.12.2010 08:35;5,95;13047;United Kingdom',
 '536370;22326;ROUND SNACK BOXES SET OF4 WOODLAND;24;1.12.2010 08:45;2,95;12583;France']

## Birim Miktarı 30dan Fazla Olanları Filtreleme

In [24]:
retailRddWithoutHeader.filter(lambda x: int(x.split(";")[3])>30).take(3)

['536378;85183B;CHARLIE & LOLA WASTEPAPER BIN FLORA;48;1.12.2010 09:37;1,25;14688;United Kingdom',
 '536381;22719;GUMBALL MONOCHROME COAT RACK;36;1.12.2010 09:41;1,06;15311;United Kingdom',
 '536384;22470;HEART OF WICKER LARGE;40;1.12.2010 09:53;2,55;18074;United Kingdom']

## Ürün isminde COFFEE geçenleri Filtreleme

In [28]:
retailRddWithoutHeader.filter(lambda x: "COFFEE" in x.split(";")[2]).take(7)

['536739;85159A;BLACK TEA,COFFEE,SUGAR JARS;2;2.12.2010 13:08;6,35;14180;United Kingdom',
 '536750;37370;RETRO COFFEE MUGS ASSORTED;6;2.12.2010 14:04;1,06;17850;United Kingdom',
 '536787;37370;RETRO COFFEE MUGS ASSORTED;6;2.12.2010 15:24;1,06;17850;United Kingdom',
 '536804;37370;RETRO COFFEE MUGS ASSORTED;72;2.12.2010 16:34;1,06;14031;United Kingdom',
 '536805;37370;RETRO COFFEE MUGS ASSORTED;12;2.12.2010 16:38;1,25;14775;United Kingdom',
 '536864;21216;SET 3 RETROSPOT TEA,COFFEE,SUGAR;1;3.12.2010 11:27;11,02;000000;United Kingdom',
 '536865;37370;RETRO COFFEE MUGS ASSORTED;1;3.12.2010 11:28;16,13;000000;United Kingdom']

## Birim Miktarı 30dan Fazla Ve İçinde COFFEE Geçenleri Filtreleme


In [29]:
def filtre(x):
    quantity = int(x.split(";")[3])
    description = x.split(";")[2]
    return (quantity>30) & ("COFFEE" in description)

In [31]:
retailRddWithoutHeader.filter(lambda x: filtre(x)).take(7)

['536804;37370;RETRO COFFEE MUGS ASSORTED;72;2.12.2010 16:34;1,06;14031;United Kingdom',
 '540176;22302;COFFEE MUG PEARS  DESIGN;36;5.01.2011 12:40;2,1;17511;United Kingdom',
 '540459;22970;LONDON BUS COFFEE MUG;36;7.01.2011 12:35;2,1;16191;United Kingdom',
 '541597;37370;RETRO COFFEE MUGS ASSORTED;216;19.01.2011 16:19;1,06;14031;United Kingdom',
 '542373;22303;COFFEE MUG APPLES DESIGN;36;27.01.2011 13:38;2,1;16767;United Kingdom',
 '542789;22970;LONDON BUS COFFEE MUG;36;1.02.2011 10:38;2,1;17511;United Kingdom',
 '543054;22300;COFFEE MUG DOG + BALL DESIGN;36;3.02.2011 10:39;2,1;13394;United Kingdom']